<a href="https://colab.research.google.com/github/vettrivikas/Survival_Analysis/blob/main/BIG_DATA_Analysis_Using_COLAB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://downloads.apache.org/spark/spark-3.0.2/spark-3.0.2-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.0.2-bin-hadoop2.7.tgz

In [5]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.2-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-02-23 07:38:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.209.129.196, 34.196.154.11, 34.234.81.220, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.209.129.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.13M  61.9MB/s    in 0.2s    

2021-02-23 07:38:28 (61.9 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://7f6739e8ef30.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://7f6739e8ef30.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [ ]:
df = spark.read.csv("/content/Files/bone-marrow-dataset.csv", header = True, inferSchema = True)

In [ ]:
df.printSchema()

root
 |-- donor_age: double (nullable = true)
 |-- donor_age_below_35: string (nullable = true)
 |-- donor_ABO: string (nullable = true)
 |-- donor_CMV: string (nullable = true)
 |-- recipient_age: double (nullable = true)
 |-- recipient_age_below_10: string (nullable = true)
 |-- recipient_age_int: string (nullable = true)
 |-- recipient_gender: string (nullable = true)
 |-- recipient_body_mass: string (nullable = true)
 |-- recipient_ABO: string (nullable = true)
 |-- recipient_rh: string (nullable = true)
 |-- recipient_CMV: string (nullable = true)
 |-- disease: string (nullable = true)
 |-- disease_group: string (nullable = true)
 |-- gender_match: string (nullable = true)
 |-- ABO_match: string (nullable = true)
 |-- CMV_status: string (nullable = true)
 |-- HLA_match: string (nullable = true)
 |-- HLA_mismatch: string (nullable = true)
 |-- antigen: string (nullable = true)
 |-- allel: string (nullable = true)
 |-- HLA_group_1: string (nullable = true)
 |-- risk_group: string (n

In [ ]:
df.show()

+---------+------------------+---------+---------+-------------+----------------------+-----------------+----------------+-------------------+-------------+------------+-------------+------------+-------------+--------------+----------+----------+---------+------------+-------+-----+-----------+----------+----------------+---------------+----------------+---------------+-----------------+------------+--------------------+------------+--------------------+--------------------+-----------------+-------------------------+----------------------+-------+-------------+---------------+
|donor_age|donor_age_below_35|donor_ABO|donor_CMV|recipient_age|recipient_age_below_10|recipient_age_int|recipient_gender|recipient_body_mass|recipient_ABO|recipient_rh|recipient_CMV|     disease|disease_group|  gender_match| ABO_match|CMV_status|HLA_match|HLA_mismatch|antigen|allel|HLA_group_1|risk_group|stem_cell_source|tx_post_relapse|CD34_x1e6_per_kg|CD3_x1e8_per_kg|CD3_to_CD34_ratio|ANC_recovery|time_to_AN

In [ ]:
df.count()

187

In [ ]:
df.select("donor_age","recipient_gender","risk_group","survival_time","survival_status").show(5)

+---------+----------------+----------+-------------+---------------+
|donor_age|recipient_gender|risk_group|survival_time|survival_status|
+---------+----------------+----------+-------------+---------------+
|22.830137|            male|      high|        999.0|              0|
|23.342466|            male|       low|        163.0|              1|
|26.394521|            male|       low|        435.0|              1|
|39.684932|          female|       low|         53.0|              1|
|33.358904|          female|      high|       2043.0|              0|
+---------+----------------+----------+-------------+---------------+
only showing top 5 rows



In [ ]:
df.select("risk_group").distinct().show()

+----------+
|risk_group|
+----------+
|       low|
|      high|
+----------+



Aggregate with Groupby

In [ ]:
from pyspark.sql import functions as F
df.groupby("risk_group").agg(F.sum("survival_time")).show()

+----------+------------------+
|risk_group|sum(survival_time)|
+----------+------------------+
|       low|          114502.0|
|      high|           61043.0|
+----------+------------------+



Counting and Removing Null values

In [ ]:
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).show()

+---------+------------------+---------+---------+-------------+----------------------+-----------------+----------------+-------------------+-------------+------------+-------------+-------+-------------+------------+---------+----------+---------+------------+-------+-----+-----------+----------+----------------+---------------+----------------+---------------+-----------------+------------+--------------------+------------+--------------------+--------------------+-----------------+-------------------------+----------------------+-------+-------------+---------------+
|donor_age|donor_age_below_35|donor_ABO|donor_CMV|recipient_age|recipient_age_below_10|recipient_age_int|recipient_gender|recipient_body_mass|recipient_ABO|recipient_rh|recipient_CMV|disease|disease_group|gender_match|ABO_match|CMV_status|HLA_match|HLA_mismatch|antigen|allel|HLA_group_1|risk_group|stem_cell_source|tx_post_relapse|CD34_x1e6_per_kg|CD3_x1e8_per_kg|CD3_to_CD34_ratio|ANC_recovery|time_to_ANC_recovery|PLT_r

In [ ]:
df.write.csv("/content/Files/simple")

In [ ]:
df.rdd.getNumPartitions()

1

In [ ]:
#Spark df to Pandas df
df_pd = df.toPandas()

In [ ]:
# Store result
df_pd.to_csv("/content/Files/pandas_preprocessed_data.csv")

In [ ]:
pwd

'/content'

In [1]:
pip install mrjob

     |████████████████████████████████| 440kB 5.7MB/s 
